In [1]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


# 重み付き平均MAPE表の作成@CG

In [2]:
date: str = "2022年8月26日"

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
target_list_process: list[int] = [256]

cg_input_list_na: list[int] = [14000, 30000, 75000, 100000]
cg_input_list_nonzer: list[int] = [11, 12, 13, 14, 15, 18]
cg_input_list_niter: list[int] = [15, 30, 75, 90]
cg_input_list_shift: list[int] = [20, 40, 60, 80, 110]

cg_target_list_na: list[int] = [1500000]
cg_target_list_nonzer: list[int] = [21]
cg_target_list_niter: list[int] = [100]
cg_target_list_shift: list[int] = [200]

input_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=input_list_process,
    list_na=cg_input_list_na,
    list_nonzer=cg_input_list_nonzer,
    list_niter=cg_input_list_niter,
    list_shift=cg_input_list_shift,
    csvDir=csvDirPath,
)
input_rawDF_cg = input_rawDF_cg.rename(columns={"Name": "functionName"})

target_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=target_list_process,
    list_na=cg_target_list_na,
    list_nonzer=cg_target_list_nonzer,
    list_niter=cg_target_list_niter,
    list_shift=cg_target_list_shift,
    csvDir=csvDirPath,
)
target_rawDF_cg = target_rawDF_cg.rename(columns={"Name": "functionName"})

cg_list_exp: list[str] = ["process", "na", "nonzer", "niter", "shift"]

list_res: list[str] = ["#Call"]

function_names: list[str] = sorted(list(set(input_rawDF_cg["functionName"])))

/tmp/ipykernel_308/3490495865.py:46: UserWarning: ./csv_files/cg_na75000_nonzer18_niter30_shift60_process4.csv is empty.
  warnings.warn(f"{filePath} is empty.")
/tmp/ipykernel_308/3490495865.py:46: UserWarning: ./csv_files/cg_na75000_nonzer15_niter90_shift80_process8.csv is empty.
  warnings.warn(f"{filePath} is empty.")


In [3]:
list_series: list[pd.Series] = []

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_cg[
        target_rawDF_cg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=cg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")

    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[cg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_cg: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

# 重み付き平均MAPE表の作成@MG

In [4]:
date: str = "2022年8月26日"

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
target_list_process: list[int] = [256]

mg_input_list_problem_size: list[int] = [32, 64, 128, 256]
mg_input_list_nit: list[int] = [4, 10, 20, 35]

mg_target_list_problem_size: list[int] = [512]
mg_target_list_nit: list[int] = [50]

# input_rawDF_cg: pd.DataFrame = return_rawDF_cg(
#     list_process=input_list_process,
#     list_na=cg_input_list_na,
#     list_nonzer=cg_input_list_nonzer,
#     list_niter=cg_input_list_niter,
#     list_shift=cg_input_list_shift,
#     csvDir=csvDirPath,
# )
# input_rawDF_cg = input_rawDF_cg.rename(columns={"Name":"functionName"})

input_rawDF_mg: pd.DataFrame = return_rawDF_mg(
    list_process=input_list_process,
    list_problem_size=mg_input_list_problem_size,
    list_nit=mg_input_list_nit,
    csvDir=csvDirPath,
)
input_rawDF_mg = input_rawDF_mg.rename(columns={"Name": "functionName"})


# target_rawDF_cg: pd.DataFrame = return_rawDF_cg(
#     list_process=target_list_process,
#     list_na=cg_target_list_na,
#     list_nonzer=cg_target_list_nonzer,
#     list_niter=cg_target_list_niter,
#     list_shift=cg_target_list_shift,
#     csvDir=csvDirPath,
# )
# target_rawDF_cg = target_rawDF_cg.rename(columns={"Name":"functionName"})

target_rawDF_mg: pd.DataFrame = return_rawDF_mg(
    list_process=target_list_process,
    list_problem_size=mg_target_list_problem_size,
    list_nit=mg_target_list_nit,
    csvDir=csvDirPath,
)
target_rawDF_mg = target_rawDF_mg.rename(columns={"Name": "functionName"})

mg_list_exp: list[str] = ["process", "problem_size", "nit"]

list_res: list[str] = ["#Call"]

function_names: list[str] = sorted(list(set(input_rawDF_mg["functionName"])))

In [5]:
list_series: list[pd.Series] = []

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_mg[
        input_rawDF_mg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_mg[
        target_rawDF_mg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=mg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")
    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[mg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_mg: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

In [6]:
retNum_mg

38.136058981254756

In [7]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


In [8]:
date: str = "2022年8月26日"


input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
target_list_process: list[int] = [256]

cg_input_list_na: list[int] = [14000, 30000, 75000, 100000]
cg_input_list_nonzer: list[int] = [11, 12, 13, 14, 15, 18]
cg_input_list_niter: list[int] = [15, 30, 75, 90]
cg_input_list_shift: list[int] = [20, 40, 60, 80, 110]

cg_target_list_na: list[int] = [1500000]
cg_target_list_nonzer: list[int] = [21]
cg_target_list_niter: list[int] = [100]
cg_target_list_shift: list[int] = [200]

input_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=input_list_process,
    list_na=cg_input_list_na,
    list_nonzer=cg_input_list_nonzer,
    list_niter=cg_input_list_niter,
    list_shift=cg_input_list_shift,
    csvDir=csvDirPath,
)

target_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=target_list_process,
    list_na=cg_target_list_na,
    list_nonzer=cg_target_list_nonzer,
    list_niter=cg_target_list_niter,
    list_shift=cg_target_list_shift,
    csvDir=csvDirPath,
)

cg_list_exp: list[str] = ["process", "na", "nonzer", "niter", "shift"]

list_res: list[str] = ["#Call"]

targetFunctionNames: list[str] = [
    "VECSET",
    "MPI_Wait()",
    "MPI_Send()",
    "SPRNVC",
    "RANDLC",
    "MPI_Irecv()",
    "ICNVRT",
]

function_names: list[str] = list(set(input_rawDF_cg["Name"].tolist()))

input_rawDF_cg = input_rawDF_cg.rename(columns={"Name": "functionName"})
target_rawDF_cg = target_rawDF_cg.rename(columns={"Name": "functionName"})


list_modelName_dict: dict[str, str] = {
    "modelIp": "線形",
    "modelLog": "対数",
    "modelLinAndIp": "線形＆反比例",
    "modelLinAndLog": "線形＆対数",
    "modelIpAndLin": "反比例＆線形",
    "modelIpAndLog": "反比例＆対数",
    "modelLogAndLin": "対数＆線形",
    "modelLogAndIp": "対数＆反比例",
    "modelProcessDividedByProblemSize": "プロセス数を問題サイズで割る",
    "modelProblemSizeDividedByProcess": "問題サイズをプロセス数で割る",
    "modelLinearSumOf2elementCombination": "全変数の組合せ積の総和",
    "modelLinearSumOfElementCombinations": "全変数の組合せ積の線形和",
    "modelLinearSumOf2elementCombinationWithSquared": "2要素の積の線形和（2乗含む）",
    "modelLinearSumOf2elementCombinationWithCubed": "2要素の積の線形和（3乗含む）",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses": "問題サイズの変数の総乗とプロセス数とをかける",
    "modelInfiniteProductOfProblemSizeDividedByProcesses": "問題サイズの変数の総乗をプロセス数で割る",
    "modelSquareRootOfProcess": "プロセス数の平方根",
    "modelSquareRootTimesOtherElems": "プロセス数の平方根と他説明変数の積",
    "modelObeyOneParameter": "一つのパラメータに依存する",
    "modelLin": "線形モデル"
    # "modelBasicTree",
}

/tmp/ipykernel_308/3490495865.py:46: UserWarning: ./csv_files/cg_na75000_nonzer18_niter30_shift60_process4.csv is empty.
  warnings.warn(f"{filePath} is empty.")
/tmp/ipykernel_308/3490495865.py:46: UserWarning: ./csv_files/cg_na75000_nonzer15_niter90_shift80_process8.csv is empty.
  warnings.warn(f"{filePath} is empty.")


In [9]:
result_series_list: list[pd.DataFrame] = []

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
    "modelInfiniteProductOfProblemSizeDividedByProcesses",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    # "modelSquareRootOfProcess",
    # "modelSquareRootTimesOtherElems",
    # "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

for function_name in function_names:
    if (function_name in targetFunctionNames) == False:
        continue
    input_rawDF_per_function: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == function_name
    ]

    models = Models(
        inputDF=input_rawDF_per_function,
        expVarColNames=cg_list_exp,
        resVarColNames=list_res,
        targetDF=None,
        modelNames=list_modelName,
    )

    models.setUpDataBeforeCalcLr()
    models.calcLr()
    models.calcMAPE()

    dictCalcedMAPE = models.returnCalculatedMAPE()

    for key in dictCalcedMAPE.keys():
        dictCalcedMAPE[key] = float(dictCalcedMAPE[key])

    dict_for_series: dict = copy.deepcopy(dictCalcedMAPE)
    dict_for_series["functionName"] = function_name

    series: pd.Series = pd.Series(dict_for_series)
    result_series_list.append(series)

resultDF: pd.DataFrame = pd.DataFrame(result_series_list)
resultDF = addLowestMAPEColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)
resultDF_before = addLowestMAPEsModelNameColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)

resultDF_before

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


,modelLin,modelIp,modelLog,modelProcessDividedByProblemSize,modelProblemSizeDividedByProcess,modelInfiniteProductOfProblemSizeDividedByProcesses,modelInfiniteProductOfProblemSizeMultipliedByProcesses,modelLinAndIp,modelLinAndLog,modelIpAndLin,modelIpAndLog,modelLogAndLin,modelLogAndIp,modelLinearSumOf2elementCombination,modelLinearSumOfElementCombinations,modelLinearSumOf2elementCombinationWithSquared,modelLinearSumOf2elementCombinationWithCubed,functionName,最低値,最適モデル
0,2.550709e+01,63.602611,43.918416,68.457739,68.707184,66.810509,66.800152,63.602603,43.918418,2.550710e+01,43.918426,2.550709e+01,63.602600,13.725285,1.353524e+01,26.548026,36.391518,ICNVRT,1.353524e+01,modelLinearSumOfElementCombinations
1,1.056736e-13,40.738362,24.184347,59.575073,95.488585,58.108038,58.101588,40.738362,24.184347,3.108404e-14,24.184348,3.033609e-14,40.738362,14.024686,2.359154e-12,19.369661,25.924923,SPRNVC,3.033609e-14,modelLogAndLin
2,3.425747e+01,52.562966,45.284269,48.827706,381.319302,61.239420,52.016990,57.396654,44.259877,4.179520e+01,45.842553,4.102758e+01,55.026591,23.419225,1.193837e+01,26.893264,33.968244,MPI_Send(),1.193837e+01,modelLinearSumOfElementCombinations
3,1.056736e-13,40.738362,24.184347,59.575073,95.488585,58.108038,58.101588,40.738362,24.184347,3.108404e-14,24.184348,3.033609e-14,40.738362,14.024686,2.359154e-12,19.369661,25.924923,VECSET,3.033609e-14,modelLogAndLin
4,3.425747e+01,52.562966,45.284269,48.827706,381.319302,61.239420,52.016990,57.396654,44.259877,4.179520e+01,45.842553,4.102758e+01,55.026591,23.419225,1.193837e+01,26.893264,33.968244,MPI_Irecv(),1.193837e+01,modelLinearSumOfElementCombinations
5,3.425747e+01,52.562966,45.284269,48.827706,381.319302,61.239420,52.016990,57.396654,44.259877,4.179520e+01,45.842553,4.102758e+01,55.026591,23.419225,1.193837e+01,26.893264,33.968244,MPI_Wait(),1.193837e+01,modelLinearSumOfElementCombinations
6,2.550716e+01,63.602569,43.918421,68.457716,68.703722,66.810487,66.800130,63.602561,43.918423,2.550717e+01,43.918430,2.550716e+01,63.602559,13.725277,1.353524e+01,26.547929,36.391398,RANDLC,1.353524e+01,modelLinearSumOfElementCombinations


In [10]:
list_series: list[pd.Series] = []

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_cg[
        target_rawDF_cg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=cg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")

    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[cg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_cg_before: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

In [11]:
print(
    resultDF_before.loc[:, ["functionName", "最低値", "最適モデル"]]
    .sort_values("functionName")
    .rename(columns={"functionName": "関数名", "最低値": "重み付きMAPE"})
    .replace(list_modelName_dict)
    .style.to_latex(label=f"{date}_resultDF_before", caption=f"（修正前）", siunitx=True)
)

print(f"retNum_cg_before = {retNum_cg_before}")

\begin{table}
\caption{（修正前）}
\label{2022年8月26日_resultDF_before}
\begin{tabular}{llSl}
{} & {関数名} & {重み付きMAPE} & {最適モデル} \\
0 & ICNVRT & 13.535235 & 全変数の組合せ積の線形和 \\
4 & MPI_Irecv() & 11.938374 & 全変数の組合せ積の線形和 \\
2 & MPI_Send() & 11.938374 & 全変数の組合せ積の線形和 \\
5 & MPI_Wait() & 11.938374 & 全変数の組合せ積の線形和 \\
6 & RANDLC & 13.535239 & 全変数の組合せ積の線形和 \\
1 & SPRNVC & 0.000000 & 対数＆線形 \\
3 & VECSET & 0.000000 & 対数＆線形 \\
\end{tabular}
\end{table}

retNum_cg_before = 7.827486063247428


In [12]:
result_series_list: list[pd.DataFrame] = []

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
    "modelInfiniteProductOfProblemSizeDividedByProcesses",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

for function_name in function_names:
    if (function_name in targetFunctionNames) == False:
        continue
    input_rawDF_per_function: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == function_name
    ]

    models = Models(
        inputDF=input_rawDF_per_function,
        expVarColNames=cg_list_exp,
        resVarColNames=list_res,
        targetDF=None,
        modelNames=list_modelName,
    )

    models.setUpDataBeforeCalcLr()
    models.calcLr()
    models.calcMAPE()

    dictCalcedMAPE = models.returnCalculatedMAPE()

    for key in dictCalcedMAPE.keys():
        dictCalcedMAPE[key] = float(dictCalcedMAPE[key])

    dict_for_series: dict = copy.deepcopy(dictCalcedMAPE)
    dict_for_series["functionName"] = function_name

    series: pd.Series = pd.Series(dict_for_series)
    result_series_list.append(series)

resultDF: pd.DataFrame = pd.DataFrame(result_series_list)
resultDF = addLowestMAPEColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)
resultDF_after = addLowestMAPEsModelNameColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)

resultDF_after

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


,modelLin,modelIp,modelLog,modelProcessDividedByProblemSize,modelProblemSizeDividedByProcess,modelInfiniteProductOfProblemSizeDividedByProcesses,modelInfiniteProductOfProblemSizeMultipliedByProcesses,modelLinAndIp,modelLinAndLog,modelIpAndLin,...,modelLinearSumOf2elementCombination,modelLinearSumOfElementCombinations,modelLinearSumOf2elementCombinationWithSquared,modelLinearSumOf2elementCombinationWithCubed,modelSquareRootOfProcess,modelSquareRootTimesOtherElems,modelObeyOneParameter,functionName,最低値,最適モデル
0,2.550709e+01,63.602611,43.918416,68.457739,68.707184,66.810509,66.800152,63.602603,43.918418,2.550710e+01,...,13.725285,1.353524e+01,26.548026,36.391518,2.550709e+01,110.789284,25.507090,ICNVRT,13.535235,modelLinearSumOfElementCombinations
1,1.056736e-13,40.738362,24.184347,59.575073,95.488585,58.108038,58.101588,40.738362,24.184347,3.108404e-14,...,14.024686,2.359154e-12,19.369661,25.924923,3.326853e-13,79.407704,0.000000,SPRNVC,0.000000,modelObeyOneParameter
2,3.425747e+01,52.562966,45.284269,48.827706,381.319302,61.239420,52.016990,57.396654,44.259877,4.179520e+01,...,23.419225,1.193837e+01,26.893264,33.968244,3.761452e+01,63.622956,34.257466,MPI_Send(),11.938374,modelLinearSumOfElementCombinations
3,1.056736e-13,40.738362,24.184347,59.575073,95.488585,58.108038,58.101588,40.738362,24.184347,3.108404e-14,...,14.024686,2.359154e-12,19.369661,25.924923,3.326853e-13,79.407704,0.000000,VECSET,0.000000,modelObeyOneParameter
4,3.425747e+01,52.562966,45.284269,48.827706,381.319302,61.239420,52.016990,57.396654,44.259877,4.179520e+01,...,23.419225,1.193837e+01,26.893264,33.968244,3.761452e+01,63.622956,34.257466,MPI_Irecv(),11.938374,modelLinearSumOfElementCombinations
5,3.425747e+01,52.562966,45.284269,48.827706,381.319302,61.239420,52.016990,57.396654,44.259877,4.179520e+01,...,23.419225,1.193837e+01,26.893264,33.968244,3.761452e+01,63.622956,34.257466,MPI_Wait(),11.938374,modelLinearSumOfElementCombinations
6,2.550716e+01,63.602569,43.918421,68.457716,68.703722,66.810487,66.800130,63.602561,43.918423,2.550717e+01,...,13.725277,1.353524e+01,26.547929,36.391398,2.550716e+01,110.789118,25.507161,RANDLC,13.535239,modelLinearSumOfElementCombinations


In [13]:
list_series: list[pd.Series] = []

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_cg[
        target_rawDF_cg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=cg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")

    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[cg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_cg_after: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

In [14]:
print(
    resultDF_after.loc[:, ["functionName", "最低値", "最適モデル"]]
    .sort_values("functionName")
    .rename(columns={"functionName": "関数名", "最低値": "重み付きMAPE"})
    .replace(list_modelName_dict)
    .style.to_latex(label=f"{date}_resultDF_before", caption=f"（修正後）", siunitx=True)
)

print(f"retNum_cg_after = {retNum_cg_after}")

\begin{table}
\caption{（修正後）}
\label{2022年8月26日_resultDF_before}
\begin{tabular}{llSl}
{} & {関数名} & {重み付きMAPE} & {最適モデル} \\
0 & ICNVRT & 13.535235 & 全変数の組合せ積の線形和 \\
4 & MPI_Irecv() & 11.938374 & 全変数の組合せ積の線形和 \\
2 & MPI_Send() & 11.938374 & 全変数の組合せ積の線形和 \\
5 & MPI_Wait() & 11.938374 & 全変数の組合せ積の線形和 \\
6 & RANDLC & 13.535239 & 全変数の組合せ積の線形和 \\
1 & SPRNVC & 0.000000 & 一つのパラメータに依存する \\
3 & VECSET & 0.000000 & 一つのパラメータに依存する \\
\end{tabular}
\end{table}

retNum_cg_after = 7.827486063247428


In [15]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


# 重み付き平均MAPEの算出@CG

In [16]:
date: str = "2022年8月26日"

input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
target_list_process: list[int] = [256]

cg_input_list_na: list[int] = [14000, 30000, 75000, 100000]
cg_input_list_nonzer: list[int] = [11, 12, 13, 14, 15, 18]
cg_input_list_niter: list[int] = [15, 30, 75, 90]
cg_input_list_shift: list[int] = [20, 40, 60, 80, 110]

cg_target_list_na: list[int] = [1500000]
cg_target_list_nonzer: list[int] = [21]
cg_target_list_niter: list[int] = [100]
cg_target_list_shift: list[int] = [200]

input_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=input_list_process,
    list_na=cg_input_list_na,
    list_nonzer=cg_input_list_nonzer,
    list_niter=cg_input_list_niter,
    list_shift=cg_input_list_shift,
    csvDir=csvDirPath,
)
input_rawDF_cg = input_rawDF_cg.rename(columns={"Name": "functionName"})

target_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=target_list_process,
    list_na=cg_target_list_na,
    list_nonzer=cg_target_list_nonzer,
    list_niter=cg_target_list_niter,
    list_shift=cg_target_list_shift,
    csvDir=csvDirPath,
)
target_rawDF_cg = target_rawDF_cg.rename(columns={"Name": "functionName"})

cg_list_exp: list[str] = ["process", "na", "nonzer", "niter", "shift"]

list_res: list[str] = ["#Call"]

function_names: list[str] = sorted(list(set(input_rawDF_cg["functionName"])))

/tmp/ipykernel_308/3490495865.py:46: UserWarning: ./csv_files/cg_na75000_nonzer18_niter30_shift60_process4.csv is empty.
  warnings.warn(f"{filePath} is empty.")
/tmp/ipykernel_308/3490495865.py:46: UserWarning: ./csv_files/cg_na75000_nonzer15_niter90_shift80_process8.csv is empty.
  warnings.warn(f"{filePath} is empty.")


In [17]:
list_series: list[pd.Series] = []

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    # "modelLinearSumOf2elementCombinationWithSquared",
    # "modelLinearSumOf2elementCombinationWithCubed",
    # "modelSquareRootOfProcess",
    # "modelSquareRootTimesOtherElems",
    # "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_cg[
        target_rawDF_cg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=cg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")

    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[cg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_cg_before: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

print(f"retNum_cg_before={retNum_cg_before}")

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

retNum_cg_before=7.827486063247428


In [18]:
list_series: list[pd.Series] = []

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_cg[
        target_rawDF_cg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=cg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")

    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[cg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_cg_after: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

print(f"retNum_cg_after={retNum_cg_after}")

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

retNum_cg_after=7.827486063247428


In [19]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


In [20]:
date: str = "2022年9月2日"

target_list_process: list[int] = [256]

cg_target_list_na: list[int] = [1500000]
cg_target_list_nonzer: list[int] = [21]
cg_target_list_niter: list[int] = [100]
cg_target_list_shift: list[int] = [200]

target_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=target_list_process,
    list_na=cg_target_list_na,
    list_niter=cg_target_list_niter,
    list_nonzer=cg_target_list_nonzer,
    list_shift=cg_target_list_shift,
    csvDir=csvDirPath,
)

target_rawDF_cg

,%Time,Exclusive,Inclusive,#Call,#Subrs,Name,process,na,nonzer,niter,shift
0,100.0,0.00616,1:44.607,1.000000e+00,1.000000e+00,.TAU_application,256,1500000,21,100,200
1,100.0,35,1:44.607,1.000000e+00,1.334000e+03,CG,256,1500000,21,100,200
2,67.1,"40,154",1:10.141,1.010000e+02,1.024140e+05,CONJ_GRAD,256,1500000,21,100,200
3,31.9,642,"33,350",1.000000e+00,3.000000e+06,MAKEA,256,1500000,21,100,200
4,30.9,"15,810","32,279",1.500000e+06,1.321420e+08,SPRNVC,256,1500000,21,100,200
5,16.7,"17,463","17,463",3.454200e+04,0.000000e+00,MPI_Wait(),256,1500000,21,100,200
6,11.7,"12,284","12,284",3.454200e+04,0.000000e+00,MPI_Send(),256,1500000,21,100,200
7,10.5,"10,999","10,999",8.809450e+07,0.000000e+00,RANDLC,256,1500000,21,100,200
8,5.2,"5,469","5,469",4.404720e+07,0.000000e+00,ICNVRT,256,1500000,21,100,200
9,0.5,0.0388,559,1.000000e+00,3.003910e+00,INITIALIZE_MPI,256,1500000,21,100,200


In [21]:
# cg_target_list_na: list[int] = [1500000]
# cg_target_list_nonzer: list[int] = [21]
# cg_target_list_niter: list[int] = [100]
# cg_target_list_shift: list[int] = [200]

get_execTime(
    benchmarkName="cg",
    condition={"process": 256, "na": 1500000, "nonzer": 21, "niter": 100, "shift": 200},
    csvDir=csvDirPath,
)

104.607

In [22]:
test_convertPprofTime()

In [23]:
target_rawDF_cg["Exclusive"]

_before_converted = list(target_rawDF_cg["Exclusive"])

_after_converted = map(convertPprofTime, _before_converted)

_after_converted

target_rawDF_cg["converted_Exclusive"] = list(_after_converted)

target_rawDF_cg

,%Time,Exclusive,Inclusive,#Call,#Subrs,Name,process,na,nonzer,niter,shift,converted_Exclusive
0,100.0,0.00616,1:44.607,1.000000e+00,1.000000e+00,.TAU_application,256,1500000,21,100,200,6.160000e-06
1,100.0,35,1:44.607,1.000000e+00,1.334000e+03,CG,256,1500000,21,100,200,3.500000e-02
2,67.1,"40,154",1:10.141,1.010000e+02,1.024140e+05,CONJ_GRAD,256,1500000,21,100,200,4.015400e+01
3,31.9,642,"33,350",1.000000e+00,3.000000e+06,MAKEA,256,1500000,21,100,200,6.420000e-01
4,30.9,"15,810","32,279",1.500000e+06,1.321420e+08,SPRNVC,256,1500000,21,100,200,1.581000e+01
5,16.7,"17,463","17,463",3.454200e+04,0.000000e+00,MPI_Wait(),256,1500000,21,100,200,1.746300e+01
6,11.7,"12,284","12,284",3.454200e+04,0.000000e+00,MPI_Send(),256,1500000,21,100,200,1.228400e+01
7,10.5,"10,999","10,999",8.809450e+07,0.000000e+00,RANDLC,256,1500000,21,100,200,1.099900e+01
8,5.2,"5,469","5,469",4.404720e+07,0.000000e+00,ICNVRT,256,1500000,21,100,200,5.469000e+00
9,0.5,0.0388,559,1.000000e+00,3.003910e+00,INITIALIZE_MPI,256,1500000,21,100,200,3.880000e-05


In [24]:
sum(target_rawDF_cg["converted_Exclusive"])

104.60017214400003

In [25]:
test_ret_relativeCost()

In [26]:
import csv


benchmarkName: str = "cg"
conditions: dict[str, list[int]] = {
    "process": [2, 4],
    "na": [14000, 75000],
    "nonzer": [11, 21],
    "niter": [90, 100],
    "shift": [60, 80],
}
ret = gen_ExtraPinputData(
    benchmarkName=benchmarkName, conditions=conditions, csvDir=csvDirPath
)
print(ret)

PARAMETER na
PARAMETER niter
PARAMETER nonzer
PARAMETER process
PARAMETER shift

POINTS ( 14000 90 11 2 60 )
POINTS ( 14000 90 11 2 80 )
POINTS ( 14000 100 11 2 60 )
POINTS ( 14000 100 11 2 80 )
POINTS ( 14000 90 21 2 60 )
POINTS ( 14000 90 21 2 80 )
POINTS ( 14000 100 21 2 60 )
POINTS ( 14000 100 21 2 80 )
POINTS ( 75000 90 11 2 60 )
POINTS ( 75000 90 11 2 80 )
POINTS ( 75000 100 11 2 60 )
POINTS ( 75000 100 11 2 80 )
POINTS ( 75000 90 21 2 60 )
POINTS ( 75000 90 21 2 80 )
POINTS ( 75000 100 21 2 60 )
POINTS ( 75000 100 21 2 80 )
POINTS ( 14000 90 11 4 60 )
POINTS ( 14000 90 11 4 80 )
POINTS ( 14000 100 11 4 60 )
POINTS ( 14000 100 11 4 80 )
POINTS ( 14000 90 21 4 60 )
POINTS ( 14000 90 21 4 80 )
POINTS ( 14000 100 21 4 60 )
POINTS ( 14000 100 21 4 80 )
POINTS ( 75000 90 11 4 60 )
POINTS ( 75000 90 11 4 80 )
POINTS ( 75000 100 11 4 60 )
POINTS ( 75000 100 11 4 80 )
POINTS ( 75000 90 21 4 60 )
POINTS ( 75000 90 21 4 80 )
POINTS ( 75000 100 21 4 60 )
POINTS ( 75000 100 21 4 80 )

REGION

In [27]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


In [28]:
# 元データによる結果を取得する
# ✅1. Extra-Pに読み込ますデータの取得
# 2. 取得できた数式から予測


input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
cg_input_list_na: list[int] = [14000, 30000, 75000, 100000]
cg_input_list_nonzer: list[int] = [11, 12, 13, 14, 15, 18]
cg_input_list_niter: list[int] = [15, 30, 75, 90]
cg_input_list_shift: list[int] = [20, 40, 60, 80, 110]

str_ExtraPinputData :str = gen_ExtraPinputData(
    benchmarkName = "cg",
    conditions={
        "process":input_list_process,
        "na":cg_input_list_na,
        "nonzer":cg_input_list_nonzer,
        "niter":cg_input_list_niter,
        "shift":cg_input_list_shift
    }
)

filePath = "./extra-p_docker/share/input_cg.txt"
with open(filePath, mode="w") as f:
    f.write(str_ExtraPinputData)


/tmp/ipykernel_308/3490495865.py:46: UserWarning: ./csv_files/cg_na75000_nonzer18_niter30_shift60_process4.csv is empty.
  warnings.warn(f"{filePath} is empty.")
/tmp/ipykernel_308/3490495865.py:46: UserWarning: ./csv_files/cg_na75000_nonzer15_niter90_shift80_process8.csv is empty.
  warnings.warn(f"{filePath} is empty.")


In [29]:
%sx pwd
%sx which extrap
%ls
%sx extrap --text /workspace/extra-p_docker/share/input_cg.txt

Dockerfile*    csv_files/           lib/                usedNote.ipynb*
Pipfile*       docker-compose.yml*  root_jupyter/       usedNote2.ipynb*
Pipfile.lock*  docs/                testing_pri.ipynb*  usedNote3.ipynb*
README.md*     draft.ipynb          testing_pub.ipynb*  usedNote4.ipynb*
app/           extra-p_docker/      txt_files/          used_scripts/


['',
 'Loading file: |          | [00:00<?]ERROR: The given file path is not valid.',
 '',
 'Loading file: |          | [00:00<?]']

In [30]:
train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48, 64]

test_lulesh_processes: list[int] = [512]
test_lulesh_iterations: list[int] = [256]
test_lulesh_sizes: list[int] = [128]

str_ExtraPinputData :str = gen_ExtraPinputData(
    benchmarkName="lulesh",
    conditions = {
        "process" : train_lulesh_processes,
        "iteration" : train_lulesh_iterations,
        "size" : train_lulesh_sizes
    },
    version=2
)

filePath = "./extra-p_docker/share/input_lulesh.txt"
with open(filePath, mode="w") as f:
    f.write(str_ExtraPinputData)


In [31]:
%sx extrap --text /workspace/extra-p_docker/share/input_lulesh.txt

['',
 'Loading file: |          | [00:00<?]ERROR: The given file path is not valid.',
 '',
 'Loading file: |          | [00:00<?]']

In [32]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


In [33]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


In [34]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello
